In [ ]:
pip install --upgrade openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import pandas as pd
import regex as re
import warnings
import numpy as np
warnings.filterwarnings("ignore")

#Prompts

In [ ]:
data=pd.read_csv('/content/all_data.csv')
data.drop(columns=[	'Unnamed: 0.1',	'Unnamed: 0'],inplace=True)
print(len(data))
data.head(2)

In [ ]:
def prepare_statement(rows):
    return f'(e{rows.e1id}:{rows.event1}) is AFTER (e{rows.e2id}:{rows.event2})' # use is AFTER and is BEFORE to create contradicting pair


In [ ]:
data['pair']=data.index
data.head(5)

,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,...,event_count_range,word_count_range,sent_count_range,relation_bool,prediction_bool,e1tense_PPF,e2tense_PPF,e1tense_PNP,e2tense_PNP,pair
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,0-5,51-75,>=2,1,0,PAST,PRESENT,PAST,NON-PAST,0
1,AQUAINT,embassies in the Kenyan and Tanzanian capitals...,2,killing,PRESPART,3,said,PAST,DIFFERENT,1,...,0-5,<=25,=1,1,0,PRESENT,PAST,NON-PAST,PAST,1
2,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,3,said,PAST,12,injured,PAST,SAME,1,...,0-5,51-75,>=2,0,0,PAST,PAST,PAST,PAST,2
3,AQUAINT,\t The American ambassador to Kenya was amon...,12,injured,PAST,4,said,PAST,SAME,2,...,0-5,<=25,=1,1,0,PAST,PAST,PAST,PAST,3
4,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,5,said,PAST,SAME,1,...,6-10,76-100,>=2,1,0,PAST,PAST,PAST,PAST,4


In [ ]:
duplicates = pd.concat([data] * 2, ignore_index=True)
duplicates

,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,...,event_count_range,word_count_range,sent_count_range,relation_bool,prediction_bool,e1tense_PPF,e2tense_PPF,e1tense_PNP,e2tense_PNP,pair
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,0-5,51-75,>=2,1,0,PAST,PRESENT,PAST,NON-PAST,0
1,AQUAINT,embassies in the Kenyan and Tanzanian capitals...,2,killing,PRESPART,3,said,PAST,DIFFERENT,1,...,0-5,<=25,=1,1,0,PRESENT,PAST,NON-PAST,PAST,1
2,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,3,said,PAST,12,injured,PAST,SAME,1,...,0-5,51-75,>=2,0,0,PAST,PAST,PAST,PAST,2
3,AQUAINT,\t The American ambassador to Kenya was amon...,12,injured,PAST,4,said,PAST,SAME,2,...,0-5,<=25,=1,1,0,PAST,PAST,PAST,PAST,3
4,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,5,said,PAST,SAME,1,...,6-10,76-100,>=2,1,0,PAST,PAST,PAST,PAST,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,TEMP3,Gingrich (e14:said) Sunday in Shanghai after (...,ei14,said,PAST,ei16,defend,FUTURE,DIFFERENT,1,...,6-10,26-50,=1,0,0,PAST,FUTURE,PAST,NON-PAST,1571
3148,TEMP3,Shen (e29:reiterated) China's hopes for a peac...,ei29,reiterated,PAST,ei30,reunification,PAST,SAME,1,...,6-10,26-50,=1,0,1,PAST,PAST,PAST,PAST,1572
3149,TEMP3,"AFP_ENG_19970401.0129TOKYO, April 1 , 1997 (AF...",ei1,applied,PAST,ei3,applied,PAST,SAME,1,...,0-5,51-75,=1,1,0,PAST,PAST,PAST,PAST,1573
3150,TEMP3,"0129TOKYO, April 1 , 1997 (AFP)Japan's top 15 ...",ei1,applied,PAST,ei2,wound,PAST,SAME,1,...,0-5,26-50,=1,0,0,PAST,PAST,PAST,PAST,1574


In [ ]:
duplicates['statement']=''

In [ ]:
# is BEFORE for first half of the dataset  and is AFTER for the second half of dataset. Thus creating a contradicting pair.
# duplicates['statement'][0:1576]=duplicates[0:1576].apply(lambda x: prepare_statement(x),
#                         axis=1) #BEFORe
duplicates['statement'][1576:]=duplicates[1576:].apply(lambda x: prepare_statement(x),
                        axis=1) #AFTER

In [ ]:
duplicates[duplicates.pair==0]

,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,...,word_count_range,sent_count_range,relation_bool,prediction_bool,e1tense_PPF,e2tense_PPF,e1tense_PNP,e2tense_PNP,pair,statement
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,51-75,>=2,1,0,PAST,PRESENT,PAST,NON-PAST,0,(e1:exploded) is BEFORE (e2:killing)
1576,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,51-75,>=2,1,0,PAST,PRESENT,PAST,NON-PAST,0,(e1:exploded) is AFTER (e2:killing)


In [ ]:
duplicates.to_csv('ExplicitTE.csv',header=True,index=False)

In [ ]:
duplicates.Relation.value_counts()

BEFORE    1828
AFTER     1324
Name: Relation, dtype: int64

In [ ]:
duplicates.Relation.iloc[2]

'AFTER'

In [ ]:
duplicates.statement.iloc[2]

'(e3:said) is BEFORE (e12:injured)'

In [ ]:
duplicates.statement.iloc[0].find(duplicates.Relation.iloc[0])

17

In [ ]:
def boolean_relation(row):
    return 'True' if row.statement.find(row.Relation) > -1 else ' False'

In [ ]:
# Entailement actual states if the statement is TRue or False based on the relation. Here for every pair there is one True statement depicting the right relation and one false statement stating the contradictory relation.
duplicates['Entailment_actual']=duplicates.apply(lambda x: boolean_relation(x),
                        axis=1)
duplicates

,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,...,sent_count_range,relation_bool,prediction_bool,e1tense_PPF,e2tense_PPF,e1tense_PNP,e2tense_PNP,pair,statement,Entailment_actual
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,>=2,1,0,PAST,PRESENT,PAST,NON-PAST,0,(e1:exploded) is BEFORE (e2:killing),True
1,AQUAINT,embassies in the Kenyan and Tanzanian capitals...,2,killing,PRESPART,3,said,PAST,DIFFERENT,1,...,=1,1,0,PRESENT,PAST,NON-PAST,PAST,1,(e2:killing) is BEFORE (e3:said),True
2,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,3,said,PAST,12,injured,PAST,SAME,1,...,>=2,0,0,PAST,PAST,PAST,PAST,2,(e3:said) is BEFORE (e12:injured),False
3,AQUAINT,\t The American ambassador to Kenya was amon...,12,injured,PAST,4,said,PAST,SAME,2,...,=1,1,0,PAST,PAST,PAST,PAST,3,(e12:injured) is BEFORE (e4:said),True
4,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,5,said,PAST,SAME,1,...,>=2,1,0,PAST,PAST,PAST,PAST,4,(e1:exploded) is BEFORE (e5:said),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,TEMP3,Gingrich (e14:said) Sunday in Shanghai after (...,ei14,said,PAST,ei16,defend,FUTURE,DIFFERENT,1,...,=1,0,0,PAST,FUTURE,PAST,NON-PAST,1571,(eei14:said) is AFTER (eei16:defend),True
3148,TEMP3,Shen (e29:reiterated) China's hopes for a peac...,ei29,reiterated,PAST,ei30,reunification,PAST,SAME,1,...,=1,0,1,PAST,PAST,PAST,PAST,1572,(eei29:reiterated) is AFTER (eei30:reunification),True
3149,TEMP3,"AFP_ENG_19970401.0129TOKYO, April 1 , 1997 (AF...",ei1,applied,PAST,ei3,applied,PAST,SAME,1,...,=1,1,0,PAST,PAST,PAST,PAST,1573,(eei1:applied) is AFTER (eei3:applied),False
3150,TEMP3,"0129TOKYO, April 1 , 1997 (AFP)Japan's top 15 ...",ei1,applied,PAST,ei2,wound,PAST,SAME,1,...,=1,0,0,PAST,PAST,PAST,PAST,1574,(eei1:applied) is AFTER (eei2:wound),True


In [ ]:
duplicates.to_csv('ExplicitTE.csv',header=True,index=False)

# Prompt GPT 3.5

In [ ]:
data=pd.read_csv('/content/ExplicitTE.csv')

In [ ]:
from openai import OpenAI
# model_name='gpt-3.5-turbo' gpt-3.5-turbo-1106
# model_name='gpt-4''gpt-4-1106-preview'
def get_completion(prompt, model='gpt-4-1106-preview'):
  client = OpenAI(api_key = '<use you API Key>')
  messages=[{"role":"user","content":prompt}]
  response=client.chat.completions.create(model=model,messages=messages,temperature=0)
  return response.choices[0].message.content

In [ ]:
prediction=[]
for index,row in data.iterrows():
    prompt="context:'{0}'.For the given context,state if the statement event:'{1}' is True or False. Give your answer in the format , Ans:<ans>".format(row.context,row.statement)
    # prompt= "context:'{0}'.\nE1:'{1}'.\nE2:'{2}'.\nFor the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>".format(row.context,row.event1,row.event2)
    print(prompt)
    response = get_completion(prompt)
    print(response)
    prediction.append(re.findall("^Ans:(.*\w*)", response)[0] if len(re.findall("^Ans:(.*\w*)", response))>0 else 'None')

0
context:'	   The American ambassador to Kenya was among hundreds (e12:injured), alocal TV (e4:said)'.
E1:'injured'.
E2:'said'.
For the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>
Relation:before


In [ ]:
data['GPT4_ET'][0:len(prediction)]=prediction
print(len(prediction))

1562


In [ ]:
data[0:len(prediction)].to_csv('Explicit_ET.csv', index=False, header=False,mode='a')

In [ ]:
st=pd.read_csv('/content/Explicit_ET.csv')
print(len(st))
st.head(2)

3146


,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,...,prediction_bool,e1tense_PPF,e2tense_PPF,e1tense_PNP,e2tense_PNP,pair,statement,Entailment_actual,GPT3_5_ET,GPT4_ET
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,...,0,PAST,PRESENT,PAST,NON-PAST,0,(e1:exploded) is BEFORE (e2:killing),True,True,True
1,AQUAINT,embassies in the Kenyan and Tanzanian capitals...,2,killing,PRESPART,3,said,PAST,DIFFERENT,1,...,0,PRESENT,PAST,NON-PAST,PAST,1,(e2:killing) is BEFORE (e3:said),True,True,False
